<a href="https://colab.research.google.com/github/m-bashari-m/vehicle-color-recognition/blob/main/src/08_xyz_analyze%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py

--2022-06-19 20:30:46--  https://raw.githubusercontent.com/m-bashari-m/vehicle-color-recognition/main/src/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12074 (12K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]  11.79K  --.-KB/s    in 0s      

2022-06-19 20:30:46 (114 MB/s) - ‘utils.py’ saved [12074/12074]



In [ ]:
from google.colab import drive
drive.mount('./drive')

Mounted at ./drive


In [ ]:
import pandas as pd
import numpy as np
import os

!pip install -q tensorflow-io
import tensorflow_io as tfio

from utils import get_train_val_ds, ErrorAnalyzer, get_class_weight, get_model

     |████████████████████████████████| 25.9 MB 1.5 MB/s 


In [ ]:
BATCH_SIZE = 64
IMG_SIZE = (256, 256)
AUTOTUNE = tf.data.AUTOTUNE
N_CLASSES = 16

In [ ]:
dataset_dir = os.path.join('drive', 'MyDrive', 'cars')
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [ ]:
_, val_ds = get_train_val_ds(train_dir, val_dir, batch_size=BATCH_SIZE, img_size=IMG_SIZE)

Found 16580 files belonging to 16 classes.
Found 3510 files belonging to 16 classes.


In [ ]:
classes, _ = get_class_weight()

In [ ]:
file_paths = val_ds.file_paths

In [ ]:
xyz_ds = (
    val_ds.
    map(lambda img, lbl: (img/255., lbl), num_parallel_calls=AUTOTUNE).
    map(lambda img, lbl: (tfio.experimental.color.rgb_to_xyz(img), lbl), num_parallel_calls=AUTOTUNE).
    prefetch(AUTOTUNE))

In [ ]:
model = get_model('xyz-model.h5')

In [ ]:
analyzer = ErrorAnalyzer(model=model,
                         ds=xyz_ds,
                         file_paths=file_paths,
                         classes=classes,
                         model_name='XYZ-model')

In [ ]:
analyzer.plot_confusion_mat()

In [ ]:
analyzer.evaluate_model()